# Notebook dedicated to compute the event rate of solar neutrinos at SNO+ for every reaction at the Sun

In [2]:
import numpy as np
import pickle

import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from scipy.integrate import simpson

from cross_section import cross_sec_ES_nu_e, integrated_sigma

# Event Rate with Oscillations

## Case of Continuous energy spectra

In [13]:
#  ======== Reactions that produces Nue and the SSM ========
continuous_spec_nu = ['pp', 'n13', 'o15', 'f17', 'b8', 'hep']
flux_values = [5.96e10, 2.80e8, 2.07e8, 5.35e6, 5.03e6, 7.95e3]  # corresponding fluxes in units of cm^-2 s^-1
ssm = 'B16_GS98_'

# Create a empty dictionary to save the expected event rate in 365 days
R_tot_cont_ev = {reac_type_i: [] for reac_type_i in continuous_spec_nu}  #Total event Rate for all flavors of neutrino
R_nue_cont_ev = {reac_type_i: [] for reac_type_i in continuous_spec_nu}  #Event rate due to electron neutrino

# =========  Directories =========
# ------- Normalized Flux Data -------
#flux_dir = 'C:/Users/joanc/jupyter notebooks/solar neutrino analysis/Flux Prediction/normalized solar spectra/data/'
#flux_fname_pattern = 'norm_spec_solar_'

flux_dir = '/home/joankl/data/solars/mc/Solar Spectra/'
flux_fname_pattern = 'norm_spec_solar_'

# ------- Pee Data -------
#Pee_dir = 'C:/Users/joanc/jupyter notebooks/solar neutrino analysis/Flux Prediction/PSelmaa out/'
#Pee_fname_pattern = 'pselmaa_test_sun_pee_'

Pee_dir = '/home/joankl/data/solars/mc/PSelmaa/'
Pee_fname_pattern = 'pselmaa_test_sun_pee_'

# ======== Loop over the Reactions ========

for i_dx, reac_i in enumerate(continuous_spec_nu):
    
    # ========= Load the Data =========
    # ---- Norm Flux ----
    with open(flux_dir + flux_fname_pattern + reac_i + '.pickle', 'rb') as handle:
        flux_data = pickle.load(handle)
    S_norm = np.array(flux_data['norm_count'])
    S_energy = np.array(flux_data['energy'])

    # ---- Pee ----
    PSelmaa_data = np.loadtxt(Pee_dir + Pee_fname_pattern + ssm + reac_i + '.txt', skiprows=1)
    Pee = PSelmaa_data[:,1]
    Pee_energy = PSelmaa_data[:,0]
    
    # ---- Flux of the corresponding reaction ----
    flux = flux_values[i_dx]
    
    # ========= Interpolation Points =========
    
    # ------- Energy Grid Definition -------
    dE = 0.01

    # === All energy range on files ===
    # Energy grid limits: Ensure that the values of the flux and Pee are within the ranges
    E_min = max(min(S_energy), min(Pee_energy))
    E_max = min(max(S_energy), max(Pee_energy))    

    # === Custom energy cut ===
    #E_min = 5
    #E_max = 15
    
    E_grid = np.arange(E_min, E_max, dE)

    # Functions that interpolates the new points to Pee and S
    f_Pee = interp1d(Pee_energy, Pee, kind='linear', bounds_error=False, fill_value=0)
    Pee_interp = f_Pee(E_grid) # New Points Evaluation

    f_S = interp1d(S_energy, S_norm, kind='linear', bounds_error=False, fill_value=0)
    S_interp = f_S(E_grid)
    
    # ========= Cross Section Calculation =========
    # Empty List to be filled with the cross sec. values for each energy of E_grid
    cross_sec_nue = []
    cross_sec_numu = []
    
    for energy_i in E_grid:
        
        cross_sec_nue.append(integrated_sigma(energy_i))
        cross_sec_numu.append(integrated_sigma(energy_i, flavor = 'numu' ))
        
        # Distinguish between cross-sec. with and without QED Corrections
        
        #if reac_i == 'b8' or reac_i == 'hep':
        #    cross_sec_nue.append(integrated_sigma(energy_i))
        #    cross_sec_numu.append(integrated_sigma(energy_i, flavor = 'numu' ))
            
        #else:
        #    cross_sec_nue.append(cross_sec_ES_nu_e(energy_i))
        #    cross_sec_numu.append(cross_sec_ES_nu_e(energy_i, flavor = 'numu' ))  
    
    cross_sec_nue = np.array(cross_sec_nue)
    cross_sec_numu = np.array(cross_sec_numu)
    
    # ========= Event Rate Calculation =========
    n_targets = 2.622e32    # N° of electrons in target material. New Value for a FV of 5.5m!
    Spec = S_interp * flux   # Flux Spectrum of the reaction
    second_to_365_days = 1/(365*24*3600)
    
    
    # --- Calculation of event rate due to all flavors ---
    integrand_function_tot = n_targets*Spec*(Pee_interp*cross_sec_nue + (1-Pee_interp)*cross_sec_numu)
    R_tot = simpson(integrand_function_tot, x = E_grid)  # Event rate in N° of events / s
    R_tot = R_tot/second_to_365_days
    
    # Save the results
    R_tot_cont_ev[reac_i] = float(R_tot)
    
    # --- Calculation of event rate due to electron neutrinos ---
    integrand_function_nue = n_targets*Spec*(Pee_interp*cross_sec_nue)
    R_nue = simpson(integrand_function_nue, x = E_grid)  # Event rate in N° of events / s
    R_nue = R_nue/second_to_365_days    
    
    # Save the results
    R_nue_cont_ev[reac_i] = float(R_nue)

In [14]:
R_nue_cont_ev

{'pp': 308946.45359988994,
 'n13': 5633.510024425803,
 'o15': 6259.6080568733605,
 'f17': 162.1812477341074,
 'b8': 862.6544916494772,
 'hep': 1.965155304030056}

In [15]:
R_tot_cont_ev

{'pp': 379960.04701587925,
 'n13': 6737.347681210744,
 'o15': 7484.320817471828,
 'f17': 193.9601910078066,
 'b8': 1151.75481007346,
 'hep': 2.5231051466089096}

In [19]:
1.965155304030056 - 2.5231051466089096

-0.5579498425788536

## Case of mono-energetic spectra

In [20]:
discrete_spec_nu = ['be7', 'pep']
flux_values = [4.85e9, 1.43e8] # corresponding fluxes in units of cm^-2 s^-1
BR_be7 = [0.10, 0.90]  # Branching Ratios the two spectral lines of Be7 that must be considered in the Rate calcualtion

ssm = 'B16_GS98_'

# Create a empty dictionary to save the expected event rate in 365 days
R_tot_disc_ev = {reac_type_i: [] for reac_type_i in discrete_spec_nu}    #Total event Rate for all flavors of neutrino
R_nue_disc_ev = {reac_type_i: [] for reac_type_i in discrete_spec_nu}  #Event rate due to electron neutrino

# =========  Directories =========
# ------- Normalized Flux Data -------
#flux_dir = 'C:/Users/Acer~/jupyter notebooks/Flux Prediction/normalized solar spectra/data/'
#flux_fname_pattern = 'norm_spec_solar_'

flux_dir = '/home/joankl/data/solars/mc/Solar Spectra/'
flux_fname_pattern = 'norm_spec_solar_'

# ------- Pee Data -------
#Pee_dir = 'C:/Users/Acer~/jupyter notebooks/Flux Prediction/PSelmaa out/'
#Pee_fname_pattern = 'pselmaa_test_sun_pee_'

Pee_dir = '/home/joankl/data/solars/mc/PSelmaa/'
Pee_fname_pattern = 'pselmaa_test_sun_pee_'

# ======== Loop over the Reactions ========

for i_dx, reac_i in enumerate(discrete_spec_nu):
    
    # ========= Load the Data =========
    # ---- Norm Flux ----
    with open(flux_dir + flux_fname_pattern + reac_i + '.pickle', 'rb') as handle:
        flux_data = pickle.load(handle)
    S_norm = np.array(flux_data['norm_count'])
    S_energy = np.array(flux_data['energy'])

    # ---- Pee ----
    PSelmaa_data = np.loadtxt(Pee_dir + Pee_fname_pattern + ssm + reac_i + '.txt', skiprows=1)
    Pee = PSelmaa_data[:,1]
    Pee_energy = PSelmaa_data[:,0]
    
    # ---- Flux of the corresponding reaction ----
    flux = flux_values[i_dx]
    
    # ========= Interpolation Points =========
    
    # ------- Energy Grid Definition -------
    #dE = 0.01
    # Energy grid limits: Ensure that the values of the flux and Pee are within the ranges
    #E_min = max(min(S_energy), min(Pee_energy))
    #E_max = min(max(S_energy), max(Pee_energy))    
    
    #E_grid = np.arange(E_min, E_max, dE)

    # Functions that interpolates the new points
    f_Pee = interp1d(Pee_energy, Pee, kind='linear', bounds_error=False, fill_value=0)
    Pee_interp = f_Pee(S_energy) # New Points Evaluation
    
    #try:
    #    f_S = interp1d(S_energy, S_norm, kind='linear', bounds_error=False, fill_value=0)
    #    S_interp = f_S(E_grid)

    #except ValueError:
    #    Pee_interp = f_Pee(S_energy)
    #    print(f'It seems that the spectrum is mono-energetic at {S_energy[0]} (MeV), then the survival probability \n will be only evaluated at the energy point. If this is not the case, review the code.')

    # ========= Cross Section Calculation =========
    # Empty List to be filled with the cross sec. values for each energy of E_grid
    cross_sec_nue = []
    cross_sec_numu = []
    
    for energy_i in S_energy:
        cross_sec_nue.append(integrated_sigma(energy_i))
        cross_sec_numu.append(integrated_sigma(energy_i, flavor = 'numu' ))
              
    cross_sec_nue = np.array(cross_sec_nue)
    cross_sec_numu = np.array(cross_sec_numu)
    
    # ========= Event Rate Calculation =========
    n_targets = 2.622e32     # N° of electrons in target material
    Spec = S_norm * flux   # Flux Spectrum of the reaction
    second_to_365_days = 1/(365*24*3600)
              
    if reac_i == 'be7':
        
        # Due to the Branching ration of the be7 reaction, the count must be done as follows for each spectral line:
        
        R1_tot = n_targets*Spec[0]*(Pee_interp[0]*cross_sec_nue[0] + (1-Pee_interp[0])*cross_sec_numu[0])
        R2_tot = n_targets*Spec[1]*(Pee_interp[1]*cross_sec_nue[1] + (1-Pee_interp[1])*cross_sec_numu[1])
        R_tot = R1_tot*BR_be7[0] + R2_tot*BR_be7[1]
        
        R1_nue = n_targets*Spec[0]*(Pee_interp[0]*cross_sec_nue[0])
        R2_nue = n_targets*Spec[1]*(Pee_interp[1]*cross_sec_nue[1])
        R_nue = R1_nue*BR_be7[0] + R2_nue*BR_be7[1]
              
    else:
        R_tot = n_targets*Spec[0]*(Pee_interp[0]*cross_sec_nue[0] + (1-Pee_interp[0])*cross_sec_numu[0])
        R_nue = n_targets*Spec[0]*(Pee_interp[0]*cross_sec_nue[0])
    
    R_tot = R_tot/second_to_365_days
    R_nue = R_nue/second_to_365_days
    
    # Save the result
    R_tot_disc_ev[reac_i] = float(R_tot)
    R_nue_disc_ev[reac_i] = float(R_nue)

In [21]:
R_nue_disc_ev

{'be7': 100540.81987555773, 'pep': 6724.41731630099}

In [22]:
R_tot_disc_ev

{'be7': 120023.96736449916, 'pep': 7973.072681691453}

In [24]:
6724.41731630099 - 7973.072681691453

-1248.655365390463

# Event Rate without Oscillations (pure $\nu_e$)

## Continuous Solar Spectra

In [9]:
#  ======== Reactions that produces Nue and the SSM ========
continuous_spec_nu = ['pp', 'n13', 'o15', 'f17', 'b8', 'hep']
flux_values = [5.96e10, 2.80e8, 2.07e8, 5.35e6, 5.03e6, 7.95e3]  # corresponding fluxes in units of cm^-2 s^-1
ssm = 'B16_GS98_'

# Create a empty dictionary to save the expected event rate in 365 days
R_nue_cont_ev = {reac_type_i: [] for reac_type_i in continuous_spec_nu}  #Event rate due to electron neutrino

# =========  Directories =========
# ------- Normalized Flux Data -------
#flux_dir = 'C:/Users/joanc/jupyter notebooks/solar neutrino analysis/Flux Prediction/normalized solar spectra/data/'
#flux_fname_pattern = 'norm_spec_solar_'

flux_dir = '/home/joankl/data/solars/mc/Solar Spectra/'
flux_fname_pattern = 'norm_spec_solar_'

# ======== Loop over the Reactions ========

for i_dx, reac_i in enumerate(continuous_spec_nu):
    
    # ========= Load the Data =========
    # ---- Norm Flux ----
    with open(flux_dir + flux_fname_pattern + reac_i + '.pickle', 'rb') as handle:
        flux_data = pickle.load(handle)
    S_norm = np.array(flux_data['norm_count'])
    S_energy = np.array(flux_data['energy'])

    # ---- Ensure positive energy ----
    S_norm = S_norm[S_energy>0]
    S_energy = S_energy[S_energy>0]
    
    # ---- Flux of the corresponding reaction ----
    flux = flux_values[i_dx]
    
    # ========= Interpolation Points =========
    
    # ------- Energy Grid Definition -------
    dE = 0.01

    # === All energy range on files ===
    # Energy grid limits: Ensure that the values of the flux and Pee are within the ranges
    E_min = min(S_energy)
    E_max = max(S_energy)    

    # === Custom energy cut ===
    #E_min = 5
    #E_max = 15
    
    E_grid = np.arange(E_min, E_max, dE)

    # Functions that interpolates the new points to Solar Spec.

    f_S = interp1d(S_energy, S_norm, kind='linear', bounds_error=False, fill_value=0)
    S_interp = f_S(E_grid)
    
    # ========= Cross Section Calculation =========
    # Empty List to be filled with the cross sec. values for each energy of E_grid
    cross_sec_nue = []
    cross_sec_numu = []
    
    for energy_i in E_grid:
        
        cross_sec_nue.append(integrated_sigma(energy_i))
        cross_sec_numu.append(integrated_sigma(energy_i, flavor = 'numu' ))
        
        # Distinguish between cross-sec. with and without QED Corrections
        
        #if reac_i == 'b8' or reac_i == 'hep':
        #    cross_sec_nue.append(integrated_sigma(energy_i))
        #    cross_sec_numu.append(integrated_sigma(energy_i, flavor = 'numu' ))
            
        #else:
        #    cross_sec_nue.append(cross_sec_ES_nu_e(energy_i))
        #    cross_sec_numu.append(cross_sec_ES_nu_e(energy_i, flavor = 'numu' ))  
    
    cross_sec_nue = np.array(cross_sec_nue)
    cross_sec_numu = np.array(cross_sec_numu)
    
    # ========= Event Rate Calculation =========
    n_targets = 2.622e32    # N° of electrons in target material. New Value for a FV of 5.5m!
    Spec = S_interp * flux   # Flux Spectrum of the reaction
    second_to_365_days = 1/(365*24*3600)
    
    
    # --- Calculation of event rate due to pure nue ---
      
    # --- Calculation of event rate due to electron neutrinos ---
    integrand_function_nue = n_targets*Spec*cross_sec_nue
    R_nue = simpson(integrand_function_nue, x = E_grid)  # Event rate in N° of events / s
    R_nue = R_nue/second_to_365_days    
    
    # Save the results
    R_nue_cont_ev[reac_i] = float(R_nue)

In [10]:
R_nue_cont_ev

{'pp': 556774.2262626854,
 'n13': 10509.665041364084,
 'o15': 12003.28189865425,
 'f17': 311.4683609423057,
 'b8': 2483.5640163962535,
 'hep': 5.657878805420771}

## Mono-energetic spectra

In [13]:
discrete_spec_nu = ['be7', 'pep']
flux_values = [4.85e9, 1.43e8] # corresponding fluxes in units of cm^-2 s^-1
BR_be7 = [0.10, 0.90]  # Branching Ratios the two spectral lines of Be7 that must be considered in the Rate calcualtion

ssm = 'B16_GS98_'

# Create a empty dictionary to save the expected event rate in 365 days
R_nue_disc_ev = {reac_type_i: [] for reac_type_i in discrete_spec_nu}  #Event rate due to electron neutrino

# =========  Directories =========
# ------- Normalized Flux Data -------
#flux_dir = 'C:/Users/Acer~/jupyter notebooks/Flux Prediction/normalized solar spectra/data/'
#flux_fname_pattern = 'norm_spec_solar_'

flux_dir = '/home/joankl/data/solars/mc/Solar Spectra/'
flux_fname_pattern = 'norm_spec_solar_'

# ======== Loop over the Reactions ========

for i_dx, reac_i in enumerate(discrete_spec_nu):
    
    # ========= Load the Data =========
    # ---- Norm Flux ----
    with open(flux_dir + flux_fname_pattern + reac_i + '.pickle', 'rb') as handle:
        flux_data = pickle.load(handle)
    S_norm = np.array(flux_data['norm_count'])
    S_energy = np.array(flux_data['energy'])

    # ---- Ensure positive energy ----
    S_norm = S_norm[S_energy>0]
    S_energy = S_energy[S_energy>0]
    
    # ---- Flux of the corresponding reaction ----
    flux = flux_values[i_dx]
    
    # ========= Cross Section Calculation =========
    # Empty List to be filled with the cross sec. values for each energy of E_grid
    cross_sec_nue = []
    cross_sec_numu = []
    
    for energy_i in S_energy:
        cross_sec_nue.append(integrated_sigma(energy_i))
        cross_sec_numu.append(integrated_sigma(energy_i, flavor = 'numu' ))
              
    cross_sec_nue = np.array(cross_sec_nue)
    cross_sec_numu = np.array(cross_sec_numu)
    
    # ========= Event Rate Calculation =========
    n_targets = 2.622e32     # N° of electrons in target material
    Spec = S_norm * flux   # Flux Spectrum of the reaction
    second_to_365_days = 1/(365*24*3600)
              
    if reac_i == 'be7':
        
        # Due to the Branching ration of the be7 reaction, the count must be done as follows for each spectral line:
        R1_nue = n_targets*Spec[0]*cross_sec_nue[0]
        R2_nue = n_targets*Spec[1]*cross_sec_nue[1]
        R_nue = R1_nue*BR_be7[0] + R2_nue*BR_be7[1]
              
    else:
        R_nue = n_targets*Spec[0]*cross_sec_nue[0]
        
    R_nue = R_nue/second_to_365_days
    
    # Save the result
    R_nue_disc_ev[reac_i] = float(R_nue)

In [14]:
R_nue_disc_ev

{'be7': 188404.82715251244, 'pep': 12864.995903126144}